## 학습정리

### 10장. 데이터 다루기
 
#### 10.1 데이터 탐색하기
* 데이터 탐색 : 가장 우선적으로 해야할 일
* 1차원 데이터 탐색하기                                                                                                     
    * 가장 간단한 경우 : 단순히 숫자로만 구성된 1차원 데이터
        1. 요약 통계치(summary statistics)계산
        2. 히스토그램 확인 : 범위를 몇 개의 구간으로 나누고 데이터의 개수를 살펴봄
        3. 
* 2차원 데이터
* 다차원 데이터

#### 10.2 namedtuple 사용하기
#### 10.3 Dataclasses
#### 10.4 정제하고 합치기
#### 10.5 데이터 처리
#### 10.6 척도 조절
#### 10.7 한편으로 : tqdm
#### 10.8 차원 축소
#### 10.9 더 공부해 보고 싶다면
